In [22]:
def nfib(n):
    def fib(k):
        if k == 0:
            return (0, 1)
        else:
            a, b = fib(k >> 1)
            c = a * ((b << 1) - a)
            d = a * a + b * b
            if k & 1:
                return (d, c + d)
            else:
                return (c, d)
    return fib(n)[0]

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.linalg as la

In [6]:
def hamSSH2_nm(N, n, m, w1, w2, u, v, a1, a2, phi, w):
    l = 2*np.pi/w
    wvec = np.array([w1, w2])
    U1 = lambda t: u*(1 + 2*np.cos(w1*t) + a1*np.cos(w2*t))*np.exp(1j*t*np.dot(m-n, wvec))
    U2 = lambda t: u*(1 - 2*np.cos(w1*t) + a1*np.cos(w2*t))*np.exp(1j*t*np.dot(m-n, wvec))
    Va = lambda t: a2*(np.cos(w1*t + phi) + a1*np.cos(w2*t + phi))*np.exp(1j*t*np.dot(m-n, wvec))
    Vb = lambda t: a2*(np.cos(w1*t + phi) - a1*np.cos(w2*t + phi))*np.exp(1j*t*np.dot(m-n, wvec))
    tvals = np.arange(0, l+0.1, 0.1)
    U1t = integrate.simpson([U1(i) for i in tvals], tvals)
    U2t = integrate.simpson([U2(i) for i in tvals], tvals)
    Vat = integrate.simpson([Va(i) for i in tvals], tvals)
    Vbt = integrate.simpson([Vb(i) for i in tvals], tvals)
                                                                          
    ham = []
    for i in range(N):
        row = []
        for j in range(N):
            if(i==j):
                m = np.array([[0, U1t],[U1t, 0]])
                row.append(m)
            elif (np.abs(i-j) == 1):
                m = np.array([[Vat, U2t],[U2t, Vbt]])
                row.append(m)
            else:
                m = np.array([[0,0],[0,0]])
                row.append(m)
        ham.append(row)
    return np.block(ham)
                                                                          
def Hf(lat, N, w1, w2, u, v, a1, a2, phi, w):
    latpose = [np.array([i,j]) for i in range(lat) for j in range(lat)]
    wvec = np.matrix(np.array([w1, w2]))
    blocks = []
    for i in latpose:
        row = []
        for j in latpose:
            if(i[0] == j[0] & i[1] == j[1]):
                row.append( hamSSH2_nm(N, i, j, w1, w2, u, v, a1, a2, phi, w) )# - np.dot(i, np.array([w1, w2]))*np.identity(2, dtype=complex) )
            else:
                row.append( hamSSH2_nm(N, i, j, w1, w2, u, v, a1, a2, phi, w) )
        blocks.append(row)
    nw = [np.dot(wvec, i) for i in latpose]
    ef = np.diagonal(nw)#, dtype=complex)
    hf = np.block(blocks) - ef
    return hf

In [56]:
lat = 1
N = 10
w2 = 0.1
fib = 5
p = nfib(fib)
q = nfib(fib-1)
w1 = (p/q)*w2
w = w2/q
u = 0.5
v = 1
a1 =1
a2 = v
phi = 10

ham = Hf(lat, N, w1, w2, u, v, a1, a2, phi, w)

In [13]:
def calculate_bott(H, Lx, Ly, ndof=2):
    L = Lx * Ly
    dim = ndof * L

    # --- Site coordinates (1D arrays) ---
    cell_x = np.zeros(dim, dtype=int)
    cell_y = np.zeros(dim, dtype=int)

    for x in range(Lx):
        for y in range(Ly):
            i = x * Ly + y
            for sub in range(ndof):
                idx = ndof * i + sub
                cell_x[idx] = x
                cell_y[idx] = y

    # --- Diagonalize Hamiltonian ---
    evals, evecs = la.eigh(H)

    # --- Projector onto occupied states ---
    occupied = evals < 0
    P = evecs[:, occupied] @ evecs[:, occupied].conj().T

    # --- Unitary position operators ---
    theta = (2.0 * np.pi / Lx) * cell_x
    phi_b = (2.0 * np.pi / Ly) * cell_y
    Ux = np.diag(np.exp(1j * theta))
    Uy = np.diag(np.exp(1j * phi_b))

    # --- Projected unitaries ---
    U = P @ Ux @ P
    V = P @ Uy @ P

    # --- Bott Operator ---
    W = V @ U @ V.conj().T @ U.conj().T

    # --- Bott index ---
    eigvals_W = la.eigvals(W)
    trace_log = np.sum(np.log(eigvals_W[np.abs(eigvals_W) > 1e-12]))
    bott_index = np.imag(trace_log) / (2 * np.pi)

    return bott_index

In [37]:
ham.shape

(20, 20)

In [57]:
calculate_bott(ham, lat, N)

-9.459856643523898e-18